<a href="https://colab.research.google.com/github/mjmousavi97/Computer-Vision/blob/main/projects/pro-003/src/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import wandb

In [2]:
wandb.init(project="VGG-flower", config={
    "epoch": 50,
    "batch_size": 16,
    "learning_rate": 0.001,
    "Architecture": "VGG",
    "pretrained": True,
    "input_size": 224,
    "optimizer": "Adam",
    })

config = wandb.config

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 1


wandb: You chose 'Create a W&B account'
wandb: Create an account here: https://wandb.ai/authorize?signup=true&ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mohammadjavad-mousavi97 (mohammadjavad-mousavi97-arak-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
# Transforms for training and validation
data_transform = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),

    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
train_dir = "/content/drive/MyDrive/Colab Notebooks/flowers/train"
val_dir = "/content/drive/MyDrive/Colab Notebooks/flowers/val"

train_dataset = datasets.ImageFolder(train_dir, transform=data_transform['train'])
val_dataset = datasets.ImageFolder(val_dir, transform=data_transform['val'])

train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False)

In [6]:
from torch._C import device
model = models.vgg16(pretrained=config.pretrained)

for param in model.features.parameters():
    param.requires_grad = False

model.classifier = nn.Sequential(
    nn.Linear(25088, 4096),
    nn.ReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(4096, 4096),
    nn.ReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(4096, 5)

)

for param in model.classifier[-1].parameters():
    param.requires_grad = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

wandb.watch(model, log="all", log_freq=10)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


100%|██████████| 528M/528M [00:05<00:00, 94.3MB/s]


In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

In [8]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10, log_every=10):
    device = next(model.parameters()).device

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for step, (images, labels) in enumerate(train_loader):
            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            optimizer.zero_grad(set_to_none=True)
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            bs = labels.size(0)
            running_loss += loss.item() * bs
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += bs

            if (step + 1) % log_every == 0:
                train_loss_step = running_loss / max(1, total)
                train_acc_step = correct / max(1, total)
                wandb.log(
                    {
                        "epoch": epoch + 1,
                        "step": epoch * len(train_loader) + (step + 1),
                        "train/loss": train_loss_step,
                        "train/acc": train_acc_step,
                        "lr": optimizer.param_groups[0]["lr"],
                    }
                )

        train_loss = running_loss / max(1, total)
        train_acc = correct / max(1, total)

        model.eval()
        val_loss_sum = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device, non_blocking=True)
                labels = labels.to(device, non_blocking=True)

                outputs = model(images)
                loss = criterion(outputs, labels)

                bs = labels.size(0)
                val_loss_sum += loss.item() * bs
                preds = outputs.argmax(dim=1)
                val_correct += (preds == labels).sum().item()
                val_total += bs

        val_loss = val_loss_sum / max(1, val_total)
        val_acc = val_correct / max(1, val_total)

        wandb.log(
            {
                "epoch": epoch + 1,
                "train/loss_epoch": train_loss,
                "train/acc_epoch": train_acc,
                "val/loss": val_loss,
                "val/acc": val_acc,
            }
        )

        print(
            f"Epoch {epoch+1}/{num_epochs} | "
            f"train_loss={train_loss:.4f} train_acc={train_acc:.4f} | "
            f"val_loss={val_loss:.4f} val_acc={val_acc:.4f}"
        )

    return model


In [ ]:
vgg16 = train_model(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=config.epoch,
    log_every=10
)

wandb.finish()
